In [1]:
%matplotlib notebook
import numpy as np
import pandas as pd
from ipywidgets import *
from IPython.display import display
from ipywidgets import widgets
from IPython.display import clear_output
import math 
import pandas as pd 
from difflib import SequenceMatcher
from itertools import groupby
from fuzzywuzzy import fuzz
import Levenshtein


In [2]:
data = pd.read_csv('Db_A.csv')
data.shape

(10188, 15)

In [3]:
data = pd.read_csv('Db_B.csv')
data.shape

(7764, 15)

In [4]:
data = pd.read_csv('Db_C.csv')
data.shape

(121, 15)

In [5]:
data = pd.read_csv('Db_D.csv')
data.shape

(4502, 15)

In [6]:
data_file_names = ['Db_A.csv', 'Db_B.csv','Db_C.csv','Db_D.csv' ]

def join_files(file_names, output_file):
    with open(output_file + '.csv', 'w') as outfile:
        for file_name in file_names:
            with open(file_name) as  infile:
                for line in infile:
                    outfile.write(line)
    return output_file + '.csv'


data_file = join_files(data_file_names, 'Db_KN')
#data = data_file
data.head()

,Clinic No.,Updated_GivenName,Updated_MiddleName,Updated_FamilyName,District,County,sub_county,Parish,village,Birthdate,Gender,Telephone Number,Treatment Supporter Telephone Number,First Encounter Date,ART Start Date
0,KSY/04333,_1115022107010205,_08011818090520,_1309121225,WAKISO,KYADONDO,NaN,MUNYONYO,SALAAMA,5/5/1987,F,704455231.0,NaN,5/29/2012,3/15/2017
1,KSY/01254,_1401110923011201,_040526091801142001,_020120011302212605,KAMPALA,KCCA,NaN,NATEETE,NANFUKA,3/1/1962,F,787250467.0,NaN,1/13/2009,5/19/2009
2,KSY/04049,_11010821140405,_19251209220901,_20211909091305,KAMPALA,KCCA,NaN,NSAMBYA,KAMWANYI,8/4/1988,F,783154640.0,772330227.0,4/4/2012,NaN
3,KSY/09781,_1401132112150715,_1908011309180108,_14011407150209,KAMPALA,KCCA,NaN,KATWE,KINYORO,3/12/1996,F,758655179.0,755183164.0,4/1/2014,8/15/2013
4,KSY/10985,_14011419051115,_0114140520,_13210701130205,KAMPALA,KCCA,MAKINDYE DIVISION,KIBUYE I,NABISAALU,5/25/1993,F,756942230.0,753813705.0,8/25/2014,8/25/2014


In [7]:
data = pd.read_csv('Db_KN.csv')
data.dropna(subset=['Clinic No.','District'],inplace=True)
data.head()

,Clinic No.,Updated_GivenName,Updated_MiddleName,Updated_FamilyName,District,County,sub_county,Parish,village,Birthdate,Gender,Telephone Number,Treatment Supporter Telephone Number,First Encounter Date,ART Start Date
0,KAW/15013,_0513130114210512,_1101021507051805,_1919051821101001,KAMPALA,KCCA,KAWEMPE DIVISION,BWAISE III,KAWAALA,11/20/1978,M,754854336,NaN,11/21/2016,2/6/2017
1,KAW/14988,_1005190301,_140107012301,_112501130201040405,KAMPALA,KCCA,NaN,BWAISE,BUKUKANYI ZN,11/5/1968,F,753691230,752447720,11/14/2016,11/28/2016
2,KAW/14972,_0618011411,_110125051409,_13012501141001,KAMPALA,KCCA,KAWEMPE DIVISION,BWAISE III,KAWAALA,7/10/1982,M,706817366,752520767,11/8/2016,1/4/2017
3,KAW/14951,_1921120109130114,_1301190518211101,_132119091909,KAMPALA,KCCA,NaN,BWAISE,BWAISE,6/26/1974,M,782265154,789817048,11/7/2016,11/21/2016
4,KAW/14841,_021805140401,_191901142521,_140111011201142609,WAKISO,KYADONDO,NaN,NASANA,GANDA,4/21/1982,F,776345644,756345444,10/13/2016,4/21/2015


In [8]:
print(len(data))
#print(data.iloc[21365])

22434


In [9]:
##STEP 5
##combine THE DATA WITH ADDRESSES WITH THE and fill in the remaining spaces with NA
##dfbc - cleanup data 
## nf - data that was omitted due to missing village names
# The idea is to get all the initial rows as per the original datasets combined 
final_data = pd.read_csv('Db_KN.csv')
final_dataframe = final_data
final_dataframe.dropna(subset=['Clinic No.','District'],inplace=True)
final_dataframe= final_dataframe.fillna("''")
final_dataframe.head()

,Clinic No.,Updated_GivenName,Updated_MiddleName,Updated_FamilyName,District,County,sub_county,Parish,village,Birthdate,Gender,Telephone Number,Treatment Supporter Telephone Number,First Encounter Date,ART Start Date
0,KAW/15013,_0513130114210512,_1101021507051805,_1919051821101001,KAMPALA,KCCA,KAWEMPE DIVISION,BWAISE III,KAWAALA,11/20/1978,M,754854336,'',11/21/2016,2/6/2017
1,KAW/14988,_1005190301,_140107012301,_112501130201040405,KAMPALA,KCCA,'',BWAISE,BUKUKANYI ZN,11/5/1968,F,753691230,752447720,11/14/2016,11/28/2016
2,KAW/14972,_0618011411,_110125051409,_13012501141001,KAMPALA,KCCA,KAWEMPE DIVISION,BWAISE III,KAWAALA,7/10/1982,M,706817366,752520767,11/8/2016,1/4/2017
3,KAW/14951,_1921120109130114,_1301190518211101,_132119091909,KAMPALA,KCCA,'',BWAISE,BWAISE,6/26/1974,M,782265154,789817048,11/7/2016,11/21/2016
4,KAW/14841,_021805140401,_191901142521,_140111011201142609,WAKISO,KYADONDO,'',NASANA,GANDA,4/21/1982,F,776345644,756345444,10/13/2016,4/21/2015


In [10]:
fd=final_dataframe
##fd["name"]= fd['Updated_FamilyName']+" "+fd['Updated_GivenName']+" "+fd['Updated_MiddleName']
fd.head()
fdx = fd
data=fd
#data=data.head(10)

In [11]:

# displaying the text widget
gn = widgets.Text(description="Given Name", width=200)
display(gn)
mn = widgets.Text(description="Middle Name", width=200)
display(mn)
fn = widgets.Text(description="Family Name", width=200)
display(fn)
d = widgets.Text(description="District", width=200)
display(d)
c = widgets.Text(description="County", width=200)
display(c)
sc = widgets.Text(description="sub_county", width=200)
display(sc)
p= widgets.Text(description="Parish", width=200)
display(p)
v = widgets.Text(description="village", width=200)
display(v)
bd = widgets.Text(description="Birth Date", width=200)
display(bd)
g = widgets.Text(description="Gender", width=200)
display(g)
tn = widgets.Text(description="Telephone Number", width=200)
display(tn)
tstn = widgets.Text(description="Treatment Support Telphone Number", width=200)
display(tstn)
fed = widgets.Text(description="First Encounter Date", width=200)
display(fed)
asd = widgets.Text(description="ART Start Date", width=200)
display(asd)

 
# preparing a container to put in created checkbox per domain
checkboxes = []
cb_container = widgets.HBox()
display(cb_container)
 
# add button that updates the graph based on the checkboxes

button = widgets.Button(description="Search", width=5000)
display(button)

def on_button_clicked(b):
    clear_output()
    #print(text.value)
    
    #selected_patient =final_dataframe[final_dataframe["Clinic No."] == "ksy/21767"].iloc[0]
    selected_patient=(gn.value,
                    mn.value,fn.value,d.value,c.value,sc.value,p.value,
                     v.value,bd.value,g.value, tn.value,tstn.value, fed.value, asd.value)
    #print(selected_patient)
    
    
        #columns (we'll use these to compute the Levenshtein distance)
    distance_columns = ['Updated_GivenName','Updated_MiddleName','Updated_FamilyName','District','County','sub_county','Parish','village' ,'Birthdate','Gender', 'Telephone Number','Treatment Supporter Telephone Number','First Encounter Date','ART Start Date']


   # data.head()
    #Calcuate the distances
    def levenshtein_distance(row):
        """
        A simple levenshtein distance function
        """
        inner_value = 0
        number =0
        #print(distance_columns)
        for k in distance_columns:
                  
            if k=='Updated_GivenName' or k=='Updated_MiddleName' or k =='Updated_FamilyName':
                constant=0.5
            elif k=='District' or k=='County' or k=='sub_county' or k=='Parish' or k=='village':
                constant=0.05
            elif k=='Birthdate' or k=='ART Start Date' or k=='First Encounter Date':
                constant=0.01
            elif k=='Telephone Number' or k=='Treatment Supporter Telephone Number':
                constant=0.075
            elif k=='Gender':
                constant=0.15

         
            inner_value += (((1 - Levenshtein.ratio(row[k], selected_patient[number]))*constant)**2)
            number += 1


        return math.sqrt(inner_value)

    # Find the distance between the selected/chosen patient and everyone else.
    levenshtein_distances = data.apply(lambda row: levenshtein_distance(row), axis=1)
    
      #STEP 5: WEIGHT SORTING 
    # Create a new dataframe with distances.
    distance_frame = pd.DataFrame(data={"dist": levenshtein_distances, "idx": levenshtein_distances.index, "Clinic No.": data["Clinic No."]})
    distance_frame.sort_values("dist", inplace=True)
    #print(distance_frame.head())
        #STEP 6: EVALUATION 
    # Find the most similar patient's to the selected patient(the lowest distance to the sp (selected patient) is sp, the second smallest is the most similar non-sp patient)
    second_smallest = distance_frame.iloc[1]["idx"]
    most_similar_to_sp = data.loc[int(second_smallest)]["Clinic No."]
    #print(distance_frame)
    print(most_similar_to_sp)

    #Get the details 
    patient = data[data["Clinic No."] == most_similar_to_sp].iloc[0]


    distance_frame = pd.DataFrame(data={"dist": levenshtein_distances, "idx": levenshtein_distances.index, "Clinic No.": data["Clinic No."], 
                                  "Updated_GivenName": data["Updated_GivenName"], "Updated_MiddleName":data["Updated_MiddleName"], "Updated_FamilyName": data["Updated_FamilyName"],"District":data["District"], "County":data["County"],"sub_county":data["sub_county"], "Parish": data["Parish"], "village":data["village"],
                                   "Birthdate":data["Birthdate"], "Gender":data["Gender"], "Telephone Number":data["Telephone Number"],"Treatment Supporter Telephone Number":data["Treatment Supporter Telephone Number"],
                                   "First Encounter Date":data["First Encounter Date"],"ART Start Date":data["ART Start Date"]})
    distance_frame.sort_values("dist", inplace=True)

    #print distances/scores just to have an overview of the differences
    print(distance_frame.head())
    
    saved_df = distance_frame.head(4)
    #-----------------categorization ---------------------------------#
    def categorize_distances(row):
        category_name = ""
        try:
            distance_value = row["dist"]
            value = float(distance_value)
            if(value)==0.000000:
                category_name = "Match"
            elif(0.000001 <= value <= 0.1):
                category_name = "Possible Match"
            elif(value > 0.1):
                category_name = "None Match"
            distance_value = ""
        except:
            category_name= "Not Categorised,Invalid Distance Value"
        return category_name
    
    categorised_distances = saved_df.apply(lambda row: categorize_distances(row), axis=1)
    saved_df = saved_df.assign(RecordCategory =categorised_distances.values)
    #saved_df['Category'] = Series(categorised_distances, index=saved_df.index)
    #categorised_distances.to_csv('saved_Levenshtein_distances_category.csv')
    #---------------end of categorization----------------------------#
    saved_df.to_csv('saved_Levenshtein_distances.csv')
    print(saved_df)
    distance_framex=distance_frame
    
button.on_click(on_button_clicked)
# function to deal with the added domain name
#button.on_click(on_button_clicked)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

...

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget